In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

In [ ]:
csv1 = pd.read_csv('high_popularity_spotify_data.csv')
csv2 = pd.read_csv('low_popularity_spotify_data.csv')

In [ ]:
csv1 = csv1.drop(columns=['track_artist', 'track_album_id', 'id', 'playlist_id', 'analysis_url','track_href', 'uri', 'type','mode','key','track_album_name','playlist_name','track_id','track_album_release_date', 'track_name','time_signature'])

In [ ]:
csv2 = csv2.drop(columns=['track_artist', 'track_album_id', 'id', 'playlist_id', 'analysis_url','track_href', 'uri', 'type','mode','key','track_album_name','playlist_name','track_id','track_album_release_date', 'track_name', 'time_signature'])

In [ ]:
csv3 = pd.concat([csv1, csv2])
csv3.columns

In [ ]:
csv3 = csv3.sample(frac=1, random_state=42).reset_index(drop=True)

In [ ]:
csv3 = pd.get_dummies(csv3, columns=['playlist_genre', 'playlist_subgenre'])
csv3 = csv3.drop(index=3068)

In [ ]:
csv3_features = csv3.copy()
csv3_labels = csv3_features.pop('track_popularity')



In [ ]:
labs = []
cont = 0
for a in csv3_features.keys():

  if cont <= 9:
    labs.append(a)
  cont +=1

In [ ]:
labs

In [ ]:
scaler = StandardScaler()
csv3_features[labs] = scaler.fit_transform(csv3_features[labs])
csv3_features = csv3_features.replace({True: 1, False: 0})


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(csv3_features, csv3_labels, test_size=0.2, random_state=42)

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(100, activation='relu', input_shape=[len(X_train.keys())]),
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(1)
])

model.compile(loss= 'mean_squared_error',
              optimizer=tf.keras.optimizers.Adam(0.1),
              metrics=['mae'])
model.fit(X_train, y_train, epochs=100)

In [ ]:
loss, accuracy = model.evaluate(X_test, y_test)

In [ ]:
indices_com_nulos = csv3.index[csv3.isnull().any(axis=1)].tolist()
print(indices_com_nulos)

In [ ]:
model.summary()

In [ ]:
predicoes = model.predict(X_test)
predicoes

In [ ]:
for true, pred in zip(y_test, predicoes):
    print(f"Valor Real: {true}, Previsão: {pred}")

In [ ]:
errors = y_test.values - predicoes

errors.mean()